## Eksportowanie modelu



In [1]:
import os
import re
from glob import glob
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.utils.data_utils import get_file
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from tqdm import tqdm
from tensorflow.keras.models import model_from_json
import json
from keras.models import Model
from tensorflow import keras
import time

2024-05-19 17:29:28.304740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 17:29:28.304877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 17:29:28.342230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-19 17:29:28.375573: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 17:29:31.140660: W tensorflow/compiler/tf2

In [2]:
#model_name = "my_ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8"
model_name = "my_ssd_resnet50_v1_fpn"
#model_name = "my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet50_v1_fpn_exported"
#model_name = "my_ssd_resnet152_v1_fpn_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet101_v1_fpn_640x640_coco17_tpu-8"
keras_model_name = f"{model_name}_keras"

images_path = f"TensorFlow/workspace/training_demo/images"
model_path = f"TensorFlow/workspace/training_demo/models/{model_name}"
exported_model_path = f"TensorFlow/workspace/training_demo/exported-models/"

keras_model_path = f"TensorFlow/workspace/training_demo/models/{keras_model_name}"
weights_path = f"{keras_model_path}/weights"
npy_weights_path = f"{keras_model_path}/npy_weights"

weights_filename = f"{keras_model_name}_weights.h5"
model_filename = f"{keras_model_name}.h5"

ckpt_dict = {"my_ssd_resnet50_v1_fpn":'/ckpt-31',
                    "my_ssd_resnet101_v1_fpn_640x640_coco17_tpu-8":"/ckpt-28",
                    "my_ssd_resnet152_v1_fpn_640x640_coco17_tpu-8":"/ckpt-26",
                    "my_ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8":"/ckpt-26",
                    "my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8": "/ckpt-51"}



### Eksportowanie wytrenowanego modelu do formatu Tensorflow
za pomocą TF Object Detection API, freezing graph

In [3]:
export_model_command = f"python TensorFlow/workspace/training_demo/exporter_main_v2.py --input_type image_tensor --pipeline_config_path {model_path}/pipeline.config --trained_checkpoint_dir {model_path}/ --output_directory {exported_model_path}/{model_name}_exported"
print(export_model_command)

python TensorFlow/workspace/training_demo/exporter_main_v2.py --input_type image_tensor --pipeline_config_path TensorFlow/workspace/training_demo/models/my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config --trained_checkpoint_dir TensorFlow/workspace/training_demo/models/my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/ --output_directory TensorFlow/workspace/training_demo/exported-models//my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8_exported


In [4]:
!{export_model_command}

/bin/bash: /home/hapek-wsl/anaconda3/envs/tf_od/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2024-05-08 20:35:46.160382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 20:35:46.160575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 20:35:46.161638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 20:35:46.167018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in oth

### Eksportowanie wytrenowanego modelu do formatu Keras

Wczytywanie modelu z TF Saved Model lub z Checkpoint

In [6]:
if model_name[-9:] == "_exported":
    print("[Loading TF2 Saved Model]")
    print(f'Loading model - {model_name}...')
    start_time = time.time()

    model = tf.saved_model.load(exported_model_path + f"{model_name}/saved_model")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'The model has been loaded - {elapsed_time:.2f}s\n')
else:
    tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)
    print("[Loading TF2 Checkpoint]")
    print(f'\nLoading model - {model_name}...')
    start_time = time.time()

    # Load pipeline config and build a detection model
    configs = config_util.get_configs_from_pipeline_file(model_path + "/pipeline.config")
    model_config = configs['model']

    model = model_builder.build(model_config=model_config, is_training=False)

    # Restore checkpoint
    ckpt = tf.compat.v2.train.Checkpoint(model=model)
    ckpt.restore(model_path + ckpt_dict[model_name]).expect_partial()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'The model has been loaded - {elapsed_time:.2f}s\n')

[Loading TF2 Saved Model]
Loading model - my_ssd_resnet50_v1_fpn_exported...
The model has been loaded - 16.17s



Ponowne zapisanie modelu w formacie TF Saved Model, obsługującym Keras

In [ ]:
tf.saved_model.save(model, exported_model_path + f"{model_name}_keras")

#### Przenoszenie modelu z Tensorflow Object Detection API na model KERAS


In [21]:
from object_detection.meta_architectures import ssd_meta_arch

class CustomSSDMetaArch(ssd_meta_arch.SSDMetaArch):
    def get_config(self):
        config = super().get_config()
        # Dodaj inne parametry konstrukcji, które są potrzebne do odtworzenia instancji.
        config.update({
            'other_arg1': self.other_arg1,
            'other_arg2': self.other_arg2,
            # Dodaj inne atrybuty tutaj
        })
        return config

In [22]:
# Ścieżka do pliku config modelu
pipeline_config = model_path + "/pipeline.config"
# Załaduj konfigurację
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)
# Zamień instancję SSDMetaArch na CustomSSDMetaArch
detection_model.__class__ = CustomSSDMetaArch

# Przywróć checkpoint modelu
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(model_path + '/' + ckpt_dict[model_name]).expect_partial()

In [23]:
class KerasObjectDetectionModel(tf.keras.Model):
    def __init__(self, detection_model, **kwargs):
        super(KerasObjectDetectionModel, self).__init__(**kwargs)
        self.detection_model = detection_model

    def call(self, inputs):
        preprocessed_inputs, shapes = self.detection_model.preprocess(inputs)
        prediction_dict = self.detection_model.predict(preprocessed_inputs, shapes)
        detections = self.detection_model.postprocess(prediction_dict, shapes)
        return detections

    def get_config(self):
        config = super(KerasObjectDetectionModel, self).get_config()
        config.update({
            "detection_model": self.detection_model
        })
        return config

In [24]:
# Utwórz instancję modelu Keras
keras_model = KerasObjectDetectionModel(detection_model)

# Określ kształt wejściowy, używając warstwy Input i Model
input_shape = (640, 640, 3)  # Dostosuj wysokość i szerokość do rozmiaru wejściowego twojego modelu
inputs = tf.keras.Input(shape=input_shape)
outputs = keras_model(inputs)

# Stwórz nowy model Keras z określonymi wejściami i wyjściami
final_keras_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Zapisz model
final_keras_model.save(exported_model_path + f"{keras_model_name}_v2")

INFO:tensorflow:Assets written to: TensorFlow/workspace/training_demo/exported-models/my_ssd_resnet50_v1_fpn_keras_v2/assets


INFO:tensorflow:Assets written to: TensorFlow/workspace/training_demo/exported-models/my_ssd_resnet50_v1_fpn_keras_v2/assets


AttributeError: 'CustomSSDMetaArch' object has no attribute 'other_arg1'

In [7]:
final_keras_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 640, 640, 3)]     0         
                                                                 
 keras_object_detection_mod  {'detection_boxes': (No   31685348  
 el (KerasObjectDetectionMo  ne, 100, 4),                        
 del)                         'detection_scores': (N             
                             one, 100),                          
                              'detection_classes': (             
                             None, 100),                         
                              'num_detections': (Non             
                             e,),                                
                              'raw_detection_boxes':             
                              (None, None, 4),                   
                              'raw_detection_scores'         

#### Extract layer weights from TF checkpoint

In [43]:
# regex for renaming the tensors to their corresponding Keras counterpart
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')


def get_filename(key):
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = filename.replace(model_name + "_", '')

    # remove "Repeat" scope from filename
    filename = re_repeat.sub('B', filename)

    if re_block8.match(filename):
        # the last block8 has different name with the previous 9 occurrences
        filename = filename.replace('Block8', 'Block8_10')
    elif filename.startswith('Logits'):
        # remove duplicate "Logits" scope
        filename = filename.replace('Logits_', '', 1)

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'

if not os.path.exists(npy_weights_path):
    os.makedirs(npy_weights_path)

reader = tf.train.load_checkpoint(exported_model_path + "/" + model_name + "_exported/checkpoint/")

for key in reader.get_variable_to_shape_map():
    # not saving the following tensors
    if key == 'global_step':
        continue
    if 'AuxLogit' in key:
        continue

    # convert tensor name into the corresponding Keras layer weight name and save 
    arr = reader.get_tensor(key)
    tensor_path = npy_weights_path + "/" + get_filename(key)
    np.save(tensor_path, arr)
    print("tensor_name: ", key)
    #print("tensor_filename: ", get_filename(key))


tensor_name:  save_counter/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-99/beta/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-96/kernel/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/moving_variance/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/moving_mean/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/gamma/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/beta/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-93/moving_mean/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-93/beta/.ATTRIBUT

#### Load weights

In [41]:
print('Instantiating an empty ResNet50V2 model...')
model = keras.applications.ResNet50V2(
    weights=None,
    input_shape= (640, 640, 3), # Rozmiar modelu 640x640
    classes = 1,
    classifier_activation= 'softmax',
)
model.summary()

Instantiating an empty ResNet50V2 model...
Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 640, 640, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 646, 646, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 320, 320, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 322, 322, 64)         0         ['conv1_conv[0][0]']          
                                              

#### Load NumPy weight files and save to a Keras HDF5 weights file

In [42]:
print('Loading numpy weights from', npy_weights_path)
weights_names = []
for layer in model.layers:
    if layer.weights:
        weights = []
        
        for w in layer.weights:
            weight_name = os.path.basename(w.name).replace(':0', '')
            weight_file = layer.name + '_' + weight_name + '.npy'
            print(f"{layer.name} || {weight_name} || {weight_file}")
            weight_arr = np.load(os.path.join(npy_weights_path, weight_file))
            weights_names.append(weight_file)
            weights.append(weight_arr)
        layer.set_weights(weights)

print('Saving weights...')
model.save_weights(os.path.join(weights_path, weights_filename))
print('Saving model...')
model.save(os.path.join(model_path, model_filename))

Loading numpy weights from TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn_keras/npy_weights
conv1_conv || kernel || conv1_conv_kernel.npy


FileNotFoundError: [Errno 2] No such file or directory: 'TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn_keras/npy_weights/conv1_conv_kernel.npy'